<a href="https://colab.research.google.com/github/BrunoReis136/API_fin_powered_by_OpenAI/blob/main/projeto_fin_gpt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# USANDO API DA FINNHUB

In [13]:
api_key = 'd53k8hhr01qkplgvn6ngd53k8hhr01qkplgvn6o0'

## Definir Classe Cliente Finnhub

In [14]:
import requests
import time

# Classe Centralizadora das operações com a API Finnhub
class FinnhubClient:

  # Construtor da Classe recebendo a chave e definindo o url base
  def __init__(self, api_key):
    self.api_key = api_key                      # Chave da API
    self.base_url = 'https://funnhub.io/api/v1' # URL base do Finnhub


  # Definir Método de busca pelo API atravéd de requisição GET
  def _get(self, endpoint, params=None):
    # Define a variável params como um dicionário
    if params is None:
      params = {}

    # Insere no params a chave de api como valor da chave 'token'
    params["token"] = self.api_key

    # Executa a requisição usando a variável 'base_url' , 'endpoint' e passando os parametros pelo dicionário 'params'
    r = requests.get(f'{self.base_url}/{endpoint}', params = params)

    # Lança excessão caso a resposta não seja 200 (erro HTTP)
    r.raise_for_status()

    # Pausa na sequencia de comandos pra folga necessária em caso de limite de execuções do API (plano gratuito)
    time.sleep(0.3)

    # Retorna o resultado do formato JSON para dicionário python
    return r.json()



  # Define método para retorno das cotação atual do ativo definido (preço, máxima, mínima, abertura, fechamento anterior)
  def quote(self, symbol):
    return self.get(
        "quote",
        {"symbol":symbol} # Código do ativo (APPL, MSFT, TSLA)
    )

  # Define método pra retorno das candles de um ativo definido em determinado período
  def candles(self, symbol, resolution, start, end):
    return self.get(
        "stock/candle",
        {
            "symbol": symbol,         # Código do ativo (APPL, MSFT, TSLA)
            "resolution": resolution, # Resolução da vela : Em minutos > 1, 5, 15, 30, 60; Período > D (days), W (weeks), M (months)
            "from": start,            # Data inicial
            "to": end                 # Data final
        }
    )


  # Define método para retorno dos rendimentos do ativo definido (real, estimado, surpresa)
  def earnings(self, symbol):
    return self.get(
        "stock/earning",
        {"symbol":symbol}
    )


  # Define método para retorno dos dados financeiros reportados (demonstração de resultados, balanço patrimonial, fluxo de caixa)
  def financials(self, symbol):
    return self.get(
        "stock/financials",
        {"symbol": symbol}
    )

## Funções de Normalização dos Dados

In [ ]:
import pandas as pd


# Funçãod e normalização dos Candles para DataFrame estruturado
def normalize_candles(data):
  # Confere se o campo "s" da resposta do API veio com valor diferente de "ok", que confirma se os dados estão válidos
  # Caso diferente, retorna um DataFrame vazio
  if data.get("s") != 'ok':
    return pd.DataFrame()

  # Cria um DataFrame a partir de cada array retornado pela API

  df = pd.Dataframe({
      "timestamp": data['t'], # Tempo em formato UNIX (epoch)
      "open":data['o'],       # Preço de abertura
      "high":data['h'],       # Preço máximo
      "low":data['l'],        # Preço mínimo
      "close":data['c'],      # Preço de fechamento
      "volume":data['v']      # Volume negociado
  })

  # Converte a coluna data para formato legível
  df['date'] = pd.to_datetime(df['timestamp'], unit='s')

  # Retorna o dataframe excluindo a coluna com tempo em formato bruto
  return df.drop(columns='timestamp')